In [ ]:
import xarray as xr
import numpy as np
import scipy.stats
import src.evt
import metpy.calc
import metpy.units
import matplotlib.pyplot as plt
import seaborn as sns
import os.path

## set plotting style
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

#### Functions

In [ ]:
def load_whoi_data():
    """Load data originally obtained from WHOI's data server"""

    ## open pre-computed PNW data
    data = xr.open_mfdataset("../data/*.nc").compute()

    return data


def get_jja(x):
    """function to subset for JJA months"""
    is_jja = (x.time.dt.month <= 8) & (x.time.dt.month >= 6)
    return x.sel(time=is_jja)

# Load and prep data

In [ ]:
## Load 2m-temperature, 2m-dewpoint, and surface pressure
data = load_whoi_data()

## Compute wetbulb temperature
Tw = metpy.calc.wet_bulb_temperature(
    pressure=data["sp"] * metpy.units.units.Pa,
    temperature=data["t2m"] * metpy.units.units.K,
    dewpoint=data["d2m"] * metpy.units.units.K,
)

## add wetbulb temp. to dataset
data["tw"] = Tw

## drop un-needed vars
data = data.drop_vars(["d2m", "sp"])

## Following Bartusek et al. (2021), compute daily mean and subset to JJA
data_daily_mean = data.resample({"time": "1D"}).mean()
data_daily_mean = get_jja(data_daily_mean)

## Compute annual max
data_annual_max = data_daily_mean.groupby("time.year").max()
year = data_annual_max.year

## Make some plots

Scatter $T_{2m}$ vs. $T_{w,2m}$

In [ ]:
##
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.set_aspect("equal")
ax.scatter(data_daily_mean.t2m, data_daily_mean.tw, s=1)
ax.set_xlabel(r"dry bulb")
ax.set_ylabel(r"wet bulb")
ax.set_xticks([280, 295])
ax.set_yticks([278, 288])
plt.show()

Timeseries of $T_{2m}$ vs. $T_{w,2m}$

In [ ]:
## should we normalize timeseries by std. dev.?
normalize = False

## functions to get max index
get_topk_idx = lambda k, var_name: np.argsort(data_annual_max[var_name]).values[-k:][
    ::-1
]
get_topk_year = lambda k, var_name: data_annual_max.year[get_topk_idx(k, var_name)]

## define preprocess function for plotting
if normalize:
    prep_fn = lambda x: (x - x.mean()) / x.std()

else:
    prep_fn = lambda x: x - x.mean()


## make the plot
fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(year, prep_fn(data_annual_max["tw"]).values, label=r"wet bulb")
ax.plot(year, prep_fn(data_annual_max["t2m"]).values, label=r"dry bulb")

## mark top-k years
topk_years = get_topk_year(k=3, var_name="t2m")
ax.scatter(topk_years, prep_fn(data_annual_max["t2m"]).sel(year=topk_years), c="k")

## label
ax.legend()
ax.set_ylabel(r"$K$")
ax.set_title(r"PNW $T_{2m}$ anomaly (daily max)")
ax.set_xticks([year[0], *topk_years.values])

plt.show()

# Compute GEV

In [ ]:
## specify model type, one of {"gev", "gp"}
model_type = "gev"
var_name = "tw"

## select variable
X = data_annual_max[var_name].values

## specify whether to hold out 2021/2022 data
hold_out_2021 = False

## training data
if hold_out_2021:
    X_train = X[:-2]

else:
    X_train = X

## select EVT distribution
if model_type == "gev":
    model_class = scipy.stats.genextreme
    thresh = None

elif model_type == "gp":
    model_class = scipy.stats.genpareto
    thresh = 318

## Empirical PDF (normalized histogram)
pdf_empirical, bin_edges = src.evt.get_empirical_pdf(X)

## Empirical return period
tr_empirical, Xr_empirical = src.evt.get_empirical_return_period(X)

## Fit model and get return levels
model = src.evt.fit_model(X_train, model_class)
Xr, tr = src.evt.get_return_levels(model, return_periods=np.logspace(0.01, 5, 100))

# ## Compute confidence interval
# bounds_fp = f"../data/PNW_ERA5/{model_type}_{var_name}_bounds.pkl"
# Xr_lb, Xr_ub = src.evt.load_return_period_bnds(
#     X, model_class=model_class, n_samples=1000, save_fp=bounds_fp
# )

## compute estimated return time for max event
tr_max = tr[np.argmin(np.abs(Xr_empirical[-1] - Xr))]

# Plot results

## PDF

In [ ]:
## test points to plot curve
X_test = np.linspace(bin_edges.min(), bin_edges.max() + 1, 100)

## setup plot
fig, ax = plt.subplots(figsize=(4, 3))

## plot empirical pdf
ax.stairs(pdf_empirical, edges=bin_edges, color="gray", fill=True, alpha=0.3)

## plot distribution fit
ax.plot(X_test, model.pdf(X_test), c="k")

## plot max value
ax.scatter(X.max(), 0, marker="x", c="r", s=50)

## label
ax.set_xlabel(r"Annual max ($K$)")
ax.set_ylabel("Prob.")

plt.show()

## Return times

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))

## plot modeled return period
ax.plot(tr, Xr, c="k")

## shade confidence bounds
# ax.fill_between(tr, Xr_ub, Xr_lb, color="k", alpha=0.1)

## plot empirical return period
ax.scatter(tr_empirical, Xr_empirical, c="r", s=10)
ax.plot(
    [tr_empirical[np.argmax(Xr_empirical)], tr_max],
    [Xr_empirical.max(), Xr_empirical.max()],
    ls="--",
    lw=1,
    c="k",
)
ax.scatter(tr_max, Xr_empirical.max(), c="k", marker="x", s=50)

## limit
ax.set_ylim([None, Xr_empirical.max() + 0.5])
ax.set_xlim([1e0, 1e3])

## label axes
ax.set_xlabel("Return period (years)")
ax.set_ylabel(r"$T_{2m}$ ($K$)")
ax.set_xscale("log")
ax.set_title(f"Est. return time: {tr_max:.0f} years")

plt.show()